# EnergyPlus Utility

## Installation From Github Repo and Setup Environment

In [ ]:
!pip uninstall -y energy-plus-utility

In [ ]:
!pip install -q "energy-plus-utility @ git+https://github.com/janithcyapa/energy-plus-utility.git@main"

In [ ]:
%load_ext autoreload
%autoreload 2

!git clone https://github.com/janithcyapa/energy-plus-utility.git
!pip install -e energy-plus-utility

In [ ]:
!cd energy-plus-utility && git pull

`prepare_colab_eplus` - Download and Install EnergyPlus 25-1 in `/root/EnergyPlus-22-1-0` 

In [3]:
import importlib.metadata
ver = importlib.metadata.version("energy-plus-utility")
print(f"\n✅ Installed 'energy-plus-utility' version: {ver}")



✅ Installed 'energy-plus-utility' version: 0.2.1+6


In [6]:
from eplus import prepare_colab_eplus
prepare_colab_eplus()

# Usage

In [ ]:
from eplus.core import EPlusUtil
import types

### Setup Basics

In [8]:
# In your main script or core.py
# verbose, 0 - no logging, 1- verbose logging, 2 - debug logging
OUT_DIR = "/simulation/eplus_out"
sim = EPlusUtil(verbose=2, out_dir=OUT_DIR)

# Reset state before setting model
sim.reset_state()
# Delete previous output directory
sim.delete_out_dir()
# Clear previous outputs
sim.clear_eplus_outputs(patterns="eplusout.*")

Initialized StateMixin
Initialized EnergyPlus State.
Initialized IDFMixin
Initialized LoggingMixin
Initialized SimulationMixin
Initialized UtilsMixin
EnergyPlus state has been reset.
Deleted output directory: /simulation/eplus_out


### Set Model using local files

In [19]:
# Specify the EnergyPlus Import
EPLUS_DIR = "/root/EnergyPlus-25-1-0"

# Define the Simulation Model
IDF = f"{EPLUS_DIR}/ExampleFiles/5ZoneAirCooled.idf"
# Select Weather Data
EPW = f"{EPLUS_DIR}/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"
# Set the Model for Simulation
sim.set_model(IDF, EPW)

EnergyPlus state has been reset.
Model set: IDF='/root/EnergyPlus-25-1-0/ExampleFiles/5ZoneAirCooled.idf', EPW='/root/EnergyPlus-25-1-0/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw', OUT_DIR='/simulation/eplus_out'


### Set Model From URL

In [9]:
url_idf = "https://raw.githubusercontent.com/janithcyapa/DHCA-Framework/refs/heads/main/System%20Models/MultiZoneOffice/MultizoneOffice.idf"
url_epw = "https://raw.githubusercontent.com/janithcyapa/DHCA-Framework/refs/heads/main/System%20Models/MultiZoneOffice/LKA_Colombo-Katunayake.434500_SWERA.epw"

# Set the Model for Simulation
sim.set_model_from_url(url_idf, url_epw)

EnergyPlus state has been reset.
Model set: IDF='/simulation/eplus_out/MultizoneOffice.idf', EPW='/simulation/eplus_out/LKA_Colombo-Katunayake.434500_SWERA.epw', OUT_DIR='/simulation/eplus_out'


### Run Basic Simualtion

In [ ]:
sim.enable_runtime_logging()
# sim.disable_runtime_logging()

In [10]:
sim.run_dry_run(include_ems_edd=False,reset=True,design_day=False)

EnergyPlus state has been reset.


0

In [11]:
sim.run_design_day()

Deleted output file: /simulation/eplus_out/eplusout.sql
Deleted output file: /simulation/eplus_out/eplusout.err
Deleted output file: /simulation/eplus_out/eplusout.audit
EnergyPlus state has been reset.


0

In [12]:
sim.run_annual()

Deleted output file: /simulation/eplus_out/eplusout.sql
Deleted output file: /simulation/eplus_out/eplusout.err
Deleted output file: /simulation/eplus_out/eplusout.audit
EnergyPlus state has been reset.


0

### Usage of Hooks

In [ ]:
# --- STEP A: DEFINE THE CUSTOM FUNCTION ---
# Notice we define it with 'self' as the first argument, just like a class method.
def dr_supervisor_logic(self, state):
    """
    A simple logic that checks the time and prints a message.
    """
    # Just printing for this demo, but you would put control logic here
    time = self.exchange.current_time(state)
    print(f"[SUPERVISOR] Checking status at time: {time:.2f}")

# --- STEP B: INJECT IT  ---
# This binds the function ONLY to this 'sim' object. 
# It creates a true method where 'self' is passed automatically.
sim.my_supervisor = types.MethodType(dr_supervisor_logic, sim)


In [ ]:
print("--- 1. REGISTERING ---")
# We register the NAME of the attribute we just attached ("my_supervisor")
registered = sim.register_handlers(
    "begin",               # Hook: Begin Timestep
    ["my_supervisor"]      # Method Name
)
print(f"Registered methods: {registered}")

print("\n--- 2. LISTING ---")
# Check what is currently scheduled for the 'begin' hook
current_list = sim.list_handlers("begin")
print(f"Handlers on 'begin' hook: {current_list}")

print("\n--- 3. DISABLING (Pausing) ---")
# Scenario: It's Winter, we don't need Demand Response. 
# We disable the hook so the logic stops running, but we don't delete it.
sim.disable_hook("begin")
print("Hook 'begin' is now DISABLED. (Simulation runs, but supervisor sleeps)")

print("\n--- 4. ENABLING (Resuming) ---")
# Scenario: Summer is back. Turn the logic back on.
sim.enable_hook("begin")
print("Hook 'begin' is now ENABLED. (Supervisor is active again)")

print("\n--- 5. UNREGISTERING (Deleting) ---")
# Scenario: We want to remove this logic entirely to replace it or clean up.
remaining = sim.unregister_handlers(
    "begin", 
    ["my_supervisor"] # Name to remove
)
print(f"Unregistered 'my_supervisor'. Remaining handlers: {remaining}")

# --- RUN ---
# sim.run_annual()

### Test

In [20]:
args = [
        '-w', EPW,
        '-d', OUT_DIR,
        IDF
    ]
    
print("Starting simulation...")
exit_code = sim.runtime.run_energyplus(sim.state, args)
    
if exit_code == 0:
    print("Simulation success!")
else:
    print("Simulation failed!")

Starting simulation...
Simulation success!


In [7]:
print(dir(sim))
print(hasattr(sim, 'run_design_day'))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_assert_out_dir_writable', '_extra_callbacks', '_log', '_orig_idf_path', '_patched_idf_path', '_runtime_log_enabled', '_runtime_log_func', 'api', 'api_version', 'clear_eplus_outputs', 'delete_out_dir', 'epw', 'exchange', 'functional', 'idf', 'out_dir', 'reset_state', 'run_annual', 'run_design_day', 'run_dry_run', 'runtime', 'set_model', 'set_model_from_url', 'state', 'state_manager', 'verbose', 'verify_api_version_match']
True
